<a href="https://colab.research.google.com/github/Stephanie9606/Python_Multimodal_Deep_Learning/blob/main/fine_tune_vgg_memes_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
# Set up the device for GPU usage
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# connect to google drive for files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Pre-processing

In [ ]:
# unzip image file
!unzip /content/drive/MyDrive/memes_image.zip

In [ ]:
memesDF = pd.read_csv('/content/drive/MyDrive/top5_memes_tidy.tsv', sep='\t')

In [ ]:
memesDF = memesDF.iloc[:, 2:9]

In [ ]:
memesDF['AltText'] = memesDF['AltText'].astype(pd.StringDtype())
memesDF['MemeLabel'] = memesDF['MemeLabel'].astype(pd.StringDtype())
memesDF['CaptionText'] = memesDF['CaptionText'].astype(pd.StringDtype())
memesDF['HashId'] = memesDF['HashId'].astype(pd.StringDtype())
memesDF['ImageURL'] = memesDF['ImageURL'].astype(pd.StringDtype())
memesDF['ImagePath'] = memesDF['ImagePath'].astype(pd.StringDtype())
memesDF['Image'] = memesDF['Image'].astype(pd.StringDtype())

In [ ]:
# Only get two label for Binary Classification
meme2cDF = memesDF[(memesDF['MemeLabel'] == "Scared Cat") | (memesDF['MemeLabel'] == "Who Killed Hannibal")]

In [ ]:
meme2cDF.head(3)

,AltText,CaptionText,ImageURL,HashId,MemeLabel,ImagePath,Image
0,Coke in Regal Cinemas > Pepsi in Regal Cinemas...,WHO WANTS A PEPSI?,http://i.imgflip.com/3xa87f.jpg,b54073fe552aa582c57a1a39e9db3f550067775516b844...,Who Killed Hannibal,/content/memes_image/3xa87f.jpg,3xa87f.jpg
1,Who Killed Hannibal | NORTH KOREA PEOPLE WITH ...,NORTH KOREA PEOPLE WITH CORONAVIRUS WE HAVE 0...,http://i.imgflip.com/3v9nm3.jpg,55a37d7aac7c769f4d517ea20b50c8ffa8e545491e45c1...,Who Killed Hannibal,/content/memes_image/3v9nm3.jpg,3v9nm3.jpg
2,Who Killed Hannibal | MINECRAFT CAVE UPDATE MI...,MINECRAFT CAVE UPDATE MINECRAFT,http://i.imgflip.com/3xjml3.jpg,ba01cf5a250ad0d9dc2901c5250ddf0e49abc8fb8e0a80...,Who Killed Hannibal,/content/memes_image/3xjml3.jpg,3xjml3.jpg


In [ ]:
# train test split
from sklearn.model_selection import train_test_split

me_trainDF, me_testDF = train_test_split(meme2cDF, test_size=0.2, random_state=15)
me_trainDF.index

Int64Index([3025,  363,  222, 3160,  100,   51, 2855, 2651,  135, 2370,
            ...
            1047,  927,  749, 3115,  221,  943,  630, 3183, 2789, 3254],
           dtype='int64', length=1861)

# Load Pre-trained Model

In [ ]:
#tf.random.set_seed(45)

# Load original model with pretrained weights from imagenet
base_model = applications.VGG16(weights="imagenet")

# Freeze base model
base_model.trainable = False

553476096/553467096 [==============================] - 12s 0us/step


In [ ]:
base_model.summary()

In [ ]:
# Model: add only dense layer -> already have good accuracy
img_height = 224
img_width = 224
inputs = Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
outputs = Dense(1, activation="sigmoid")(x) # binary: "sigmoid", multiclass: "softmax"
model = Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 1000)              138357544 
                                                                 
 dense_1 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 138,358,545
Trainable params: 1,001
Non-trainable params: 138,357,544
_________________________________________________________________


In [ ]:
# Try deeper model on top
img_height = 224
img_width = 224
inputs = Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)

hidden1 = Flatten(name="flatten")(x)
hidden2 = Dense(512, activation="relu")(hidden1)
hidden3 = Dropout(0.5)(hidden2)

outputs = Dense(1, activation="sigmoid")(hidden3)
model = Model(inputs, outputs)
model.summary()

In [ ]:
base_model.layers.pop()
model2 = Model(base_model.input, base_model.layers[-2].output)
model2.summary()
model2 = model2(inputs, training=False)

In [ ]:
# pop base_model to 4096 output feature layer + deeper model
base_model.layers.pop()
model2 = Model(base_model.input, base_model.layers[-2].output)
#model2.summary()
x = model2(inputs, training=False)

hidden1 = Flatten(name="flatten")(x)
hidden2 = Dense(512, activation="relu")(hidden1)
hidden3 = Dropout(0.5)(hidden2)

outputs = Dense(1, activation="sigmoid")(hidden3) # output probabilities for both classes
model = Model(inputs, outputs)
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 model_8 (Functional)        (None, 4096)              134260544 
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense_7 (Dense)             (None, 512)               2097664   
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                 513       
                                                                 
Total params: 136,358,721
Trainable params: 2,098,177
Non-t

In [ ]:
 # Loading training data
img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(preprocessing_function=applications.vgg16.preprocess_input, rescale=1./255.,validation_split=0.25)
train_generator = train_datagen.flow_from_dataframe(dataframe=me_trainDF, 
                                                    directory=None, 
                                                    x_col="ImagePath", 
                                                    y_col="MemeLabel", 
                                                    subset="training", 
                                                    seed = 55, 
                                                    class_mode="binary", 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size)

valid_generator = train_datagen.flow_from_dataframe(dataframe=me_trainDF, 
                                                    directory=None, 
                                                    x_col="ImagePath", 
                                                    y_col="MemeLabel", 
                                                    subset="validation", 
                                                    seed = 55, 
                                                    class_mode="binary", 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size)

print(f"Class labels: {train_generator.class_indices}")

#step_size = train_generator.n//train_generator.batch_size

Found 1396 validated image filenames belonging to 2 classes.
Found 465 validated image filenames belonging to 2 classes.
Class labels: {'Scared Cat': 0, 'Who Killed Hannibal': 1}


# Hyperparameter tuning

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
from keras.layers.core import Dense, Dropout, Activation
import sys

In [ ]:
# Hyperparameter tuning
def create_model(params):
  img_height = 224
  img_width = 224
  inputs = Input(shape=(img_height, img_width, 3))

  base_model = applications.VGG16(weights="imagenet")
#  base_model.layers.pop()
  base_model = Model(base_model.input, base_model.layers[-2].output)
  x = base_model(inputs, training=False)
  hidden1 = Dense(params['units1'], activation = 'relu')(x)
  hidden2 = Dropout(params['dropout1'])(hidden1)

  outputs = Dense(1, activation = 'sigmoid')(hidden2) # sigmoid, softmax...

  model = Model(inputs, outputs)
  return model

In [ ]:
x = train_generator
validation_data = valid_generator

# Define NN
def f_nn(params):   
    model = create_model(params)
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

    model.fit(x, validation_data = validation_data, epochs=10, batch_size=32, verbose = 2)
#    model.fit(x, epochs=params['num_epochs'], batch_size=params['batch_size'], verbose = 2)

    loss, acc = model.evaluate(validation_data, verbose = 0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model} 

# Search space
space = {'units1': hp.uniform('units1', 64,1024),
         'dropout1': hp.uniform('dropout1', .25,.75)
#         'batch_size' : hp.uniform('batch_size', 28,128)
        }

# Trials
trials = Trials()

best = fmin(f_nn, space, algo=tpe.suggest, max_evals=8, trials=trials) # max_eval=?
print('best: ', best)

Epoch 1/10

44/44 - 61s - loss: 3.6637 - accuracy: 0.5208 - val_loss: 0.3867 - val_accuracy: 0.5140 - 61s/epoch - 1s/step

Epoch 2/10

44/44 - 44s - loss: 0.1461 - accuracy: 0.5208 - val_loss: 0.0682 - val_accuracy: 0.5140 - 44s/epoch - 1s/step

Epoch 3/10

44/44 - 44s - loss: 0.2542 - accuracy: 0.5208 - val_loss: 0.1072 - val_accuracy: 0.5140 - 44s/epoch - 999ms/step

Epoch 4/10

44/44 - 44s - loss: 0.0691 - accuracy: 0.5208 - val_loss: 0.0462 - val_accuracy: 0.5140 - 44s/epoch - 998ms/step

Epoch 5/10

44/44 - 44s - loss: 1.7284 - accuracy: 0.5208 - val_loss: 0.1119 - val_accuracy: 0.5140 - 44s/epoch - 994ms/step

Epoch 6/10

44/44 - 44s - loss: 0.1700 - accuracy: 0.5208 - val_loss: 0.0715 - val_accuracy: 0.5140 - 44s/epoch - 992ms/step

Epoch 7/10

44/44 - 43s - loss: 0.1395 - accuracy: 0.5208 - val_loss: 0.0526 - val_accuracy: 0.5140 - 43s/epoch - 988ms/step

Epoch 8/10

44/44 - 44s - loss: 0.1080 - accuracy: 0.5208 - val_loss: 0.0262 - val_accuracy: 0.5140 - 44s/epoch - 993ms/step

ResourceExhaustedError: ignored

In [ ]:
# Manual train with fixed configuration from Fine-tune
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

model.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.005), metrics=["accuracy"])

model.fit(train_generator,
          steps_per_epoch=step_size_train,
          validation_data=valid_generator,
          validation_steps=step_size_valid,
          epochs=10,
          verbose=2 # show: epoch 1/10
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
43/43 - 22s - loss: 1.2907 - accuracy: 0.9142 - val_loss: 1.7696e-05 - val_accuracy: 1.0000 - 22s/epoch - 520ms/step
Epoch 2/10
43/43 - 24s - loss: 0.0262 - accuracy: 0.9934 - val_loss: 6.7651e-06 - val_accuracy: 1.0000 - 24s/epoch - 554ms/step
Epoch 3/10
43/43 - 21s - loss: 0.0172 - accuracy: 0.9978 - val_loss: 5.9692e-06 - val_accuracy: 1.0000 - 21s/epoch - 491ms/step
Epoch 4/10
43/43 - 21s - loss: 0.0178 - accuracy: 0.9985 - val_loss: 0.0028 - val_accuracy: 0.9978 - 21s/epoch - 494ms/step
Epoch 5/10
43/43 - 21s - loss: 0.0011 - accuracy: 0.9993 - val_loss: 1.6182e-05 - val_accuracy: 1.0000 - 21s/epoch - 491ms/step
Epoch 6/10
43/43 - 22s - loss: 0.0141 - accuracy: 0.9978 - val_loss: 0.0052 - val_accuracy: 0.9978 - 22s/epoch - 507ms/step
Epoch 7/10
43/43 - 22s - loss: 0.0078 - accuracy: 0.9978 - val_loss: 0.0046 - val_accuracy: 0.9978 - 22s/epoch - 511ms/step
Epoch 8/10
43/43 - 21s - loss: 0.0146 - accuracy: 0.9963 - val_loss: 6.9979e-07 - val_accuracy: 1.0000 - 21s/epoch -

# Original Code before Fine-tune

In [ ]:
!pip install mlflow

In [ ]:
import mlflow.tensorflow

In [ ]:
# Check out the MLflow UI as this runs
mlflow.tensorflow.autolog(every_n_iter=2)

In [ ]:
# optimizer and metric
model.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.005), metrics=["accuracy"]) 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Train the model 
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

model.fit(train_generator,
          steps_per_epoch=step_size_train,
          validation_data=valid_generator,
          validation_steps=step_size_valid,
          epochs=10,
          verbose=2 # show: epoch 1/10
)

2022/03/27 20:08:04 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9d596fa92ef545fd832512bc0ae3ab89', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/10
43/43 - 40s - loss: 1.5578 - accuracy: 0.5183 - val_loss: 1.3773e-04 - val_accuracy: 0.5112 - 40s/epoch - 932ms/step
Epoch 2/10
43/43 - 21s - loss: 0.0346 - accuracy: 0.5191 - val_loss: 8.7123e-05 - val_accuracy: 0.5067 - 21s/epoch - 488ms/step
Epoch 3/10
43/43 - 22s - loss: 0.0337 - accuracy: 0.5205 - val_loss: 1.7694e-04 - val_accuracy: 0.5179 - 22s/epoch - 512ms/step
Epoch 4/10
43/43 - 26s - loss: 0.0073 - accuracy: 0.5198 - val_loss: 0.0029 - val_accuracy: 0.5223 - 26s/epoch - 605ms/step
Epoch 5/10
43/43 - 21s - loss: 0.0099 - accuracy: 0.5213 - val_loss: 3.4809e-04 - val_accuracy: 0.5112 - 21s/epoch - 490ms/step
Epoch 6/10
43/43 - 21s - loss: 0.0073 - accuracy: 0.5198 - val_loss: 0.0038 - val_accuracy: 0.5134 - 21s/epoch - 491ms/step
Epoch 7/10
43/43 - 26s - loss: 0.0146 - accuracy: 0.5198 - val_loss: 4.9030e-05 - val_accuracy: 0.5201 - 26s/epoch - 612ms/step
Epoch 8/10
43/43 - 21s - loss: 0.0078 - accuracy: 0.5198 - val_loss: 1.2050e-04 - val_accuracy: 0.5179 - 21s/epo

2022/03/27 20:13:39 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.2+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). MLflow logged a pip requirement for this package as 'jaxlib==0.3.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


In [ ]:
# Evaluate model on test set
test_datagen = ImageDataGenerator(preprocessing_function=applications.vgg16.preprocess_input, rescale=1./255.)

In [ ]:
# Evaluate
batch_size = 32

test_generator = test_datagen.flow_from_dataframe(
  dataframe=me_testDF, 
  directory=None, 
  x_col="ImagePath", 
  y_col="MemeLabel", 
  class_mode="binary", 
  target_size=(img_height, img_width),
  shuffle=False,
  batch_size=batch_size
)

step_size_test = test_generator.n//test_generator.batch_size

eval_results = model.evaluate(test_generator, steps=step_size_test)
print(f"Loss: {eval_results[0]}. Accuracy: {eval_results[1]}")

Found 466 validated image filenames belonging to 2 classes.
14/14 [==============================] - 6s 379ms/step - loss: 5.9871e-05 - accuracy: 1.0000
Loss: 5.987134500173852e-05. Accuracy: 1.0


In [ ]:
# Predict
np.set_printoptions(suppress=True)
predict_results = model.predict(test_generator, steps=step_size_test)
print(predict_results)